In [ ]:
import os
import sys
import numpy as np
from matplotlib import cm
import open3d as o3d
import carla
import math
import cv2
import weakref
from sklearn.cluster import DBSCAN
from scipy.spatial import KDTree
import cv2

import constants

In [ ]:
cloud_point = []

In [ ]:
class PointCloudProcessor:
    """
    Processes point cloud data frames, performs clustering, tracking, and vehicle classification.
    """
    def __init__(self, eps=constants.EPS, min_samples=constants.MIN_SAMPLES):
        self.eps = eps
        self.min_samples = min_samples

        # Initialize tracking history and IDs
        self.previous_centroids = []
        self.previous_cluster_ids = []
        self.cluster_history = []
        self.tracking_id = 0
        self.cluster_color_map = {}
        self.previous_centroids_map = {}
        self.previous_directions_map = {}
        self.vehicle_type_cluster = {}

        self.first = True

    def process_frames(self, cloud_point_frame):
        """Processes each frame to perform clustering, tracking, and vehicle classification."""
       
        current_frame = cloud_point_frame
        # Perform clustering
        clustering = DBSCAN(eps=self.eps, min_samples=self.min_samples).fit(current_frame)
        labels = clustering.labels_

        current_centroids = []
        cluster_points = []

        # Extract clusters and centroids
        for cluster_label in set(labels):
            if cluster_label == -1:
                continue  # Skip noise points
            
            cluster_data = current_frame[labels == cluster_label]
  
            if np.any(cluster_data[:, 2] >= constants.CLUSTER_UPPER_EXCLUDE):
                continue
            
            clus_points = current_frame[labels == cluster_label]
            cluster_points.append(clus_points)

            cluster_pcd = o3d.geometry.PointCloud()
            cluster_pcd.points = o3d.utility.Vector3dVector(clus_points)

            bbox = cluster_pcd.get_axis_aligned_bounding_box()
            bbox_center = bbox.get_center()
            current_centroids.append(bbox_center)

        current_centroids = np.array(current_centroids)

        # Track clusters
        if self.first == True:
            # For the first frame, initialize the cluster IDs
            cluster_ids = np.arange(len(current_centroids))
            self.tracking_id = len(current_centroids)
            self.first = False
        else:
            cluster_ids, self.tracking_id = self.track_clusters(current_centroids, self.previous_centroids, self.previous_cluster_ids)

        # Save the cluster history for visualization
        self.cluster_history.clear()
        self.cluster_history.append((cluster_points, current_centroids, cluster_ids))

        # Update previous centroids and cluster IDs
        self.previous_centroids = current_centroids
        self.previous_cluster_ids = cluster_ids

        # Assign colors to clusters
        for cluster_id in cluster_ids:
            self.assign_color(cluster_id)
        
    def track_clusters(self, current_centroids, previous_centroids, prev_cluster_ids, threshold=constants.DIST_THRESHOLD_TRACKING):
        """Matches current centroids with previous centroids using nearest neighbor approach."""
        if len(previous_centroids) == 0:
            return np.arange(len(current_centroids)) + self.tracking_id, self.tracking_id + len(current_centroids)
        
        # Create a KDTree for fast nearest neighbor lookup
        tree = KDTree(previous_centroids)
        
        # Assign new tracking IDs based on nearest neighbors from the previous frame
        assigned_ids = np.full(len(current_centroids), -1)  # -1 for new clusters
        distances, indices = tree.query(current_centroids)
        
        # Assign tracking IDs based on distance threshold
        for i, (dist, idx) in enumerate(zip(distances, indices)):
            if dist < threshold:
                assigned_ids[i] = prev_cluster_ids[idx]  # Match with existing cluster ID
        
        # Assign new IDs to unmatched clusters
        new_clusters = (assigned_ids == -1)
        assigned_ids[new_clusters] = np.arange(self.tracking_id, self.tracking_id + np.sum(new_clusters))
        
        return assigned_ids, self.tracking_id + np.sum(new_clusters)
    
    def classify_vehicle(self, bbox):
        """Classifies the object as a car, truck, or motorcycle based on the bounding box dimensions."""
        dimensions = bbox.get_extent() 
        dimensions = sorted(dimensions, reverse=True)

        d0, d1, d2 = dimensions[0], dimensions[1], dimensions[2]
    
        for vehicle_type, size_range in self.vehicle_size_ranges.items():
            if (size_range['0'][0] <= d0 <= size_range['0'][1] and
                size_range['1'][0] <= d1 <= size_range['1'][1] and
                size_range['2'][0] <= d2 <= size_range['2'][1]):
                return vehicle_type
        return None

    def assign_color(self, cluster_id):
        """Assigns a color to a cluster if not already assigned."""
        if cluster_id not in self.cluster_color_map:
            color_index = len(self.cluster_color_map) % len(constants.COLOR_NAMES)  # Cycle through the color list
            self.cluster_color_map[cluster_id] = (constants.COLOR_NAMES[color_index], constants.COLOR_VALUES[constants.COLOR_NAMES[color_index]])
        
    def create_arrow(self, start, end, bbox, previous_direction=None, scale_factor=constants.ARROW_SCALE_FACTOR):
        """
        Creates an arrow mesh representing movement direction with a scaled length based on the magnitude of the vector.
        Uses Exponential Moving Average (EMA) for smoothing the direction.
        """
        kmh = self.find_velocity_kmh(end, start)
        
        if kmh > constants.VELOCITY_THRESHOLD:
            # Calculate the direction vector
            new_direction = end - start
            new_direction[2] = 0  # Ignore Z-axis movement for 2D plane movement visualization
    
            # If there is a previous direction, update it using exponential moving average
            if previous_direction is not None:
                updated_direction = constants.PREVIOUS_DIRECTION_PARAM * previous_direction + constants.UPDATED_DIRECTION_PARAM * new_direction  # Apply EMA
            else:
                updated_direction = new_direction
    
            # Calculate the magnitude (length) of the direction vector
            length = np.linalg.norm(updated_direction)
    
            if length == 0:
                return None  # No arrow needed if there's no movement
    
            # Scale the arrow length by a given factor
            scaled_length = length * scale_factor
    
            # Normalize the updated direction vector
            updated_direction_normalized = updated_direction / length
    
            # Create an arrow mesh with length proportional to the scaled movement
            arrow1 = o3d.geometry.TriangleMesh.create_arrow(cylinder_radius=0.1, cone_radius=0.2,
                                                           cylinder_height=scaled_length * 0.3, cone_height=scaled_length * 0.2)
            arrow2 = o3d.geometry.TriangleMesh.create_arrow(cylinder_radius=0.1, cone_radius=0.2,
                                                           cylinder_height=scaled_length * 0.3, cone_height=scaled_length * 0.2)
    
            # Align the arrow with the updated direction vector
            z_unit_vector = np.array([0, 0, 1])  # Upward vector
            axis = np.cross(z_unit_vector, updated_direction_normalized)
            axis_length = np.linalg.norm(axis)
    
            if axis_length != 0:
                axis /= axis_length  # Normalize the axis of rotation
                angle = np.arccos(np.clip(np.dot(z_unit_vector, updated_direction_normalized), -1.0, 1.0))
                R = o3d.geometry.get_rotation_matrix_from_axis_angle(axis * angle)
                arrow1.rotate(R, center=(0, 0, 0))
                arrow2.rotate(R, center=(0, 0, 0))

            front_top_center, back_top_center = self.find_top_edge_centers(bbox)
            # Translate the arrow to start from the previous centroid (start position)
            arrow1.translate(front_top_center)
            arrow2.translate(back_top_center)
            arrow1.paint_uniform_color([1, 0, 1])  # Set arrow color to purple
            arrow2.paint_uniform_color([1, 0, 1])  # Set arrow color to purple
    
            return arrow1, arrow2
        
        return None

    def find_velocity_kmh(self, start, end, time=1/20):
        """Computes velocity in km/h between two points."""
        v = np.linalg.norm(end - start) / time
        kmh = int(3.6 * v)
        return kmh
    
    def find_top_edge_centers(self, bbox):
        bbox_corners = np.asarray(bbox.get_box_points())
        top_face_points = bbox_corners[bbox_corners[:, 2] == np.max(bbox_corners[:, 2])]
        edges = {
            ('0', '1'): np.linalg.norm(top_face_points[0] - top_face_points[1]),
            ('0', '2'): np.linalg.norm(top_face_points[0] - top_face_points[2]),
            ('0', '3'): np.linalg.norm(top_face_points[0] - top_face_points[3]),
            ('1', '2'): np.linalg.norm(top_face_points[1] - top_face_points[2]),
            ('1', '3'): np.linalg.norm(top_face_points[1] - top_face_points[3]),
            ('2', '3'): np.linalg.norm(top_face_points[2] - top_face_points[3])
        }

        sorted_edges = sorted(edges.items(), key=lambda x: x[1], reverse=True)
        a, b = sorted_edges[4][0]
        c, d = sorted_edges[5][0]
        front = (top_face_points[int(a)]+top_face_points[int(b)])/2
        front[2] = top_face_points[int(a)][2]
        back = (top_face_points[int(c)]+top_face_points[int(d)])/2
        back[2] = top_face_points[int(c)][2]
        return front, back

class VehicleManager:
    def __init__(self, world, blueprint_library, spawn_transform, vehicle_bp_filter=constants.CAR_MODEL):
        self.world = world
        self.blueprint_library = blueprint_library
        self.spawn_transform = spawn_transform
        self.vehicle_bp_filter = vehicle_bp_filter
        self.vehicle_bp = blueprint_library.filter(vehicle_bp_filter)[0]
        self.vehicle = self.world.spawn_actor(self.vehicle_bp, self.spawn_transform)
        self.original_transform = spawn_transform
        self.collision_sensor = None
        self.collision_detected = False
        self.collision_location = None
        self.attach_collision_sensor()

    def attach_collision_sensor(self):
        collision_bp = self.blueprint_library.find('sensor.other.collision')
        self.collision_sensor = self.world.spawn_actor(collision_bp, carla.Transform(), attach_to=self.vehicle)
        weak_self = weakref.ref(self)
        self.collision_sensor.listen(lambda event: VehicleManager._on_collision(weak_self, event))

    @staticmethod
    def _on_collision(weak_self, event):
        self = weak_self()
        if not self:
            return
        self.collision_detected = True
        self.collision_location = event.transform.location
        # You can process other collision details here if needed.

    def apply_control(self, throttle, steer):
        self.vehicle.apply_control(carla.VehicleControl(throttle=throttle, steer=steer))

    def respawn(self):
        if self.collision_sensor:
            self.collision_sensor.stop()
            self.collision_sensor.destroy()
            self.collision_sensor = None
        if self.vehicle:
            self.vehicle.destroy()
            self.vehicle = None
        # Respawn vehicle
        self.vehicle = self.world.try_spawn_actor(self.vehicle_bp, self.original_transform)
        if self.vehicle is None:
            print("Failed to respawn vehicle.")
            return False
            
        self.attach_collision_sensor()
        return True

    def destroy(self):
        if self.lidar:
            self.lidar.destroy()
            self.lidar = None
        if self.collision_sensor:
            self.collision_sensor.stop()
            self.collision_sensor.destroy()
            self.collision_sensor = None
        if self.vehicle:
            self.vehicle.destroy()
            self.vehicle = None

class LidarManager:
    def __init__(self, world, blueprint_library, vehicle):
        self.world = world
        self.vehicle = vehicle
        self.point_list = o3d.geometry.PointCloud()
        self.cloud_point_data = []
        self.cloud_point = []
        self.lidar_bp = blueprint_library.find('sensor.lidar.ray_cast')
        self.lidar_bp.set_attribute('upper_fov', str(constants.LIDAR_UPPER_FOV))
        self.lidar_bp.set_attribute('lower_fov', str(constants.LIDAR_LOWER_FOV))
        self.lidar_bp.set_attribute('channels', str(constants.LIDAR_CHANNELS))
        self.lidar_bp.set_attribute('range', str(constants.LIDAR_RANGE))
        self.lidar_bp.set_attribute('rotation_frequency', str(constants.LIDAR_ROTATION_FREQUENCY))
        self.lidar_bp.set_attribute('points_per_second', str(constants.LIDAR_POINTS_PER_SECOND))
        self.lidar_transform = carla.Transform(carla.Location(x=0, z=constants.LIDAR_POSITION_HEIGHT))
        
        self.lidar = self.world.spawn_actor(self.lidar_bp, self.lidar_transform, attach_to=self.vehicle)
        
        self.lidar.listen(lambda data: self.lidar_callback(data))

    def lidar_callback(self, point_cloud):
        """Processes LiDAR data and updates the point cloud."""
        data = np.copy(np.frombuffer(point_cloud.raw_data, dtype=np.dtype('f4')))
        data = np.reshape(data, (int(data.shape[0] / 4), 4))

        # Isolate the intensity and compute a color for it
        intensity = data[:, -1]
        intensity_col = 1.0 - np.log(intensity) / np.log(np.exp(-0.004 * 100))
        int_color = np.c_[
            np.interp(intensity_col, constants.VID_RANGE, constants.VIRIDIS[:, 0]),
            np.interp(intensity_col, constants.VID_RANGE, constants.VIRIDIS[:, 1]),
            np.interp(intensity_col, constants.VID_RANGE, constants.VIRIDIS[:, 2])]

        points = data[:, :-1]
        points[:, :1] = -points[:, :1]

        # Apply filters for height
        height_filter = points[:, 2] >= -1.7
        height_filter_1 = points[:, 2] <= 2
        valid_points = np.logical_and(height_filter, height_filter_1)

        # Filter points and colors
        filtered_points = points[valid_points]
        filtered_colors = int_color[valid_points]

        # Update the point cloud data
        self.point_list.points = o3d.utility.Vector3dVector(filtered_points)
        self.point_list.colors = o3d.utility.Vector3dVector(filtered_colors)

        self.cloud_point_data.clear()
        self.cloud_point_data.append(filtered_points)
        self.cloud_point.append(filtered_points)
        cloud_point.append(filtered_points)

    def destroy(self):
        if self.lidar:
            self.lidar.destroy()

            self.lidar = None
                
def maintain_speed(s):
    """Function to maintain the desired speed."""
    if s >= constants.PREFERRED_SPEED:
        return 0
    elif s < constants.PREFERRED_SPEED - constants.SPEED_THRESHOLD:
        return 0.9  # High throttle
    else:
        return 0.4  # Moderate throttle



In [ ]:
client = carla.Client('localhost', constants.PORT)
client.set_timeout(constants.TIMEOUT_MAX)
world = client.get_world()

# Set up the simulation settings
original_settings = world.get_settings()
settings = world.get_settings()
traffic_manager = client.get_trafficmanager(8000)
traffic_manager.set_synchronous_mode(True)

settings.fixed_delta_seconds = constants.DELTA
settings.synchronous_mode = True
settings.no_rendering_mode = False
world.apply_settings(settings)

# Get the blueprint library
blueprint_library = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()
start_point = spawn_points[0]

# Spawn vehicle1 with LiDAR
new_spawn_transform1 = carla.Transform(
    carla.Location(
        start_point.location.x,
        start_point.location.y,
        start_point.location.z
    ),
    start_point.rotation
)
desired_rotation = carla.Rotation(pitch=0.0, yaw=90.0, roll=0.0)
new_spawn_transform2 = carla.Transform(
        carla.Location(
            start_point.location.x + 15,
            start_point.location.y - 10,
            start_point.location.z
        ),
        desired_rotation
)

vehicle_manager1 = VehicleManager(world=world,blueprint_library=blueprint_library, spawn_transform=new_spawn_transform1)
vehicle_manager2 = VehicleManager(world=world,blueprint_library=blueprint_library, spawn_transform=new_spawn_transform2)

# Set up the Open3D visualizer
vis = o3d.visualization.Visualizer()
vis.create_window(
    window_name='Carla LiDAR',
    width=960,
    height=540,
    left=480,
    top=270)
vis.get_render_option().background_color = [0.05, 0.05, 0.05]
vis.get_render_option().point_size = 1
vis.get_render_option().show_coordinate_frame = True

# Access the ViewControl object
view_ctl = vis.get_view_control()

processor = PointCloudProcessor(eps=constants.EPS, min_samples=constants.MIN_SAMPLES)

frame = 0

In [ ]:
try:
    while True:
        world.tick()

        if cv2.waitKey(1) == ord('q'):
            break

        # Vehicle control for vehicle1
        steering_angle = 0
        v = vehicle_manager1.vehicle.get_velocity()
        speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2), 0)
        estimated_throttle = maintain_speed(speed)
        vehicle_manager1.apply_control(estimated_throttle, steering_angle)

        # Vehicle control for vehicle2
        v = vehicle_manager2.vehicle.get_velocity()
        speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2), 0)
        estimated_throttle = maintain_speed(speed)
        vehicle_manager2.apply_control(estimated_throttle, steering_angle)

        # Collision handling
        if vehicle_manager1.collision_detected:
            # Destroy and respawn vehicles
            vehicle_manager1.respawn()
            vehicle_manager2.respawn()
            # Reset collision detection flag
            vehicle_manager1.collision_detected = False
            vehicle_manager1.collision_location = None
            # Reattach LiDAR manager to new vehicle
            lidar_manager.destroy()
            lidar_manager = LidarManager(world, blueprint_library, vehicle_manager1.vehicle)

        # LiDAR processing and visualization
        if frame == 2:
            vis.add_geometry(lidar_manager.point_list)

        if lidar_manager.cloud_point_data:
            # Process the latest point cloud data
            processor.process_frames(lidar_manager.cloud_point_data[0])
            cluster_points, centroids, cluster_ids = processor.cluster_history[0]

            all_points = []
            all_colors = []
            bounding_boxes = []
            centroids_spheres = []
            arrows = []

            for i, cluster in enumerate(cluster_points):
                # Add all points to the list for visualization
                all_points.append(cluster)

                # Create an Open3D point cloud for the current cluster
                cluster_pcd = o3d.geometry.PointCloud()
                cluster_pcd.points = o3d.utility.Vector3dVector(cluster)

                # Compute the bounding box for the cluster
                bbox = cluster_pcd.get_axis_aligned_bounding_box()
                bbox_center = bbox.get_center()

                # Ensure that the color is assigned
                processor.assign_color(cluster_ids[i])

                # Use bounding box center as centroid and classify for vehicle types
                vehicle_type = processor.classify_vehicle(bbox)
                
                if (cluster_ids[i] in processor.vehicle_type_cluster) and ((vehicle_type == None) or vehicle_type == processor.vehicle_type_cluster[cluster_ids[i]]):
                    vehicle_type = processor.vehicle_type_cluster[cluster_ids[i]]
                elif (cluster_ids[i] in processor.vehicle_type_cluster) and (vehicle_type != None) and vehicle_type != processor.vehicle_type_cluster[cluster_ids[i]]:
                    processor.vehicle_type_cluster[cluster_ids[i]] = vehicle_type

                if vehicle_type:
                    # Add the bounding box to the list for visualization
                    bbox.color = (0, 1, 0)  # Green
                    bounding_boxes.append(bbox)

                    centroid_color = processor.vehicle_centroid_colors[vehicle_type]  # Color based on vehicle type

                    # Set the color of the entire point cloud cluster to match the centroid color
                    all_colors.append(np.tile(centroid_color, (len(cluster), 1)))  # Apply centroid color to the points

                    # Check if we have a previous centroid for this cluster to draw an arrow
                    if cluster_ids[i] in processor.previous_centroids_map:
                        prev_centroid = processor.previous_centroids_map[cluster_ids[i]]

                        # Get the previous direction if it exists
                        previous_direction = processor.previous_directions_map.get(cluster_ids[i], None)

                        # Create and append the arrow with EMA
                        arrow = processor.create_arrow(prev_centroid, bbox_center, previous_direction)
                        if arrow:
                            arrows.append(arrow)

                        # Store the new direction for future EMA updates
                        new_direction = bbox_center - prev_centroid
                        processor.previous_directions_map[cluster_ids[i]] = new_direction  # Update the direction map

                    # Update the previous centroid for this cluster
                    processor.previous_centroids_map[cluster_ids[i]] = bbox_center

                else:
                    centroid_color = [0, 0, 1]  # Default blue
                    all_colors.append(np.tile(centroid_color, (len(cluster), 1)))  # Apply default color to points

                # Create a sphere at the centroid
                sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.1)
                sphere.translate(bbox_center)
                sphere.paint_uniform_color(centroid_color)
                centroids_spheres.append(sphere)

            if len(all_points) > 0:
                # Flatten the points and colors arrays
                all_points = np.vstack(all_points)
                all_colors = np.vstack(all_colors)

                # Create an Open3D point cloud object for visualization
                pcd = o3d.geometry.PointCloud()
                pcd.points = o3d.utility.Vector3dVector(all_points)
                pcd.colors = o3d.utility.Vector3dVector(all_colors)

            vis.clear_geometries()
            entities_to_draw = [pcd] + bounding_boxes + arrows + centroids_spheres

            for entity in entities_to_draw:
                vis.add_geometry(entity)

            view_ctl.set_zoom(constants.ZOOM_LIDAR_BIRD_EYE_PARAM)
            vis.poll_events()
            vis.update_renderer()

        # Check if the window is closed by the user
        if not vis.poll_events():
            print("Window closed by user. Terminating...")
            break

        frame += 1

except KeyboardInterrupt:
    print("Interrupted by user. Terminating...")

finally:
    # Clean up
    world.apply_settings(original_settings)
    traffic_manager.set_synchronous_mode(False)

    if vehicle_manager1:
        vehicle_manager1.destroy()
    if vehicle_manager2:
        vehicle_manager2.destroy()
    if lidar_manager:
        lidar_manager.destroy()
    vis.destroy_window()